In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import cv2
import numpy as np
from easydict import EasyDict as edict
import tensorflow.contrib.slim as slim
from tensorflow.python import pywrap_tensorflow
from google.colab import drive
drive.mount('/content/drive')
#_____________________________________________________________________________________________________________#
__C = edict()
cfg = __C
__C.DIR = '/content/drive/My Drive/VSimplE/'
__C.POOLING_SIZE = 7
__C.VRD_BATCH_NUM = 30
__C.VRD_NUM_CLASS = 101
__C.VRD_NUM_RELA = 70
__C.VRD_TRAIN_ROUND = 20
__C.VRD_BATCH_NUM_RELA = 50
__C.VRD_AU_PAIR = 5
__C.VRD_IOU_TRAIN = 0.5
__C.VRD_IOU_TEST = 0.5
__C.IM_SIZE = 600
__C.IM_MAX_SIZE = 1000
__C.TRAIN = edict()
__C.RESNET = edict()
__C.RESNET.FIXED_BLOCKS = 1
__C.VTR = edict()
#_______________________________________________________________________________________________________#
def rela_recall(test_roidb, pred_roidb, N_recall):
	N_right = 0.0
	N_total = 0.0
	N_data = len(test_roidb)
	num_right = np.zeros([N_data, ])
	for i in range(N_data):
		rela_gt = test_roidb[i]['rela_gt']
		if len(rela_gt) == 0:
			continue
		sub_gt = test_roidb[i]['sub_gt']
		obj_gt = test_roidb[i]['obj_gt']
		sub_box_gt = test_roidb[i]['sub_box_gt']
		obj_box_gt = test_roidb[i]['obj_box_gt']

		pred_rela = pred_roidb[i]['pred_rela']
		pred_rela_score = pred_roidb[i]['pred_rela_score']
		sub_dete = pred_roidb[i]['sub_dete']
		obj_dete = pred_roidb[i]['obj_dete']
		sub_box_dete = pred_roidb[i]['sub_box_dete']
		obj_box_dete = pred_roidb[i]['obj_box_dete']

		N_rela = len(rela_gt)
		N_total = N_total + N_rela

		N_pred = len(pred_rela)

		sort_score = np.sort(pred_rela_score)[::-1]
		if N_recall >= N_pred:
			thresh = -1
		else:
			thresh = sort_score[N_recall]

		detected_gt = np.zeros([N_rela, ])
		for j in range(N_pred):
			if pred_rela_score[j] <= thresh:
				continue

			for k in range(N_rela):
				if detected_gt[k] == 1:
					continue
				if (sub_gt[k] == sub_dete[j]) and (obj_gt[k] == obj_dete[j]) and (rela_gt[k] == pred_rela[j]):
					s_iou = compute_iou_each(sub_box_dete[j], sub_box_gt[k])
					o_iou = compute_iou_each(obj_box_dete[j], obj_box_gt[k])
					if (s_iou >= 0.5) and (o_iou >= 0.5):
						detected_gt[k] = 1
						N_right = N_right + 1
						num_right[i] = num_right[i] + 1

	acc = N_right / N_total
	print('Recall: @{0}	>>>	Number right predicion : {1} ' .format(N_recall, int(N_right) ))
	print('Number of total image  : ' ,int(N_total) )
	return acc, num_right

def read_roidb(roidb_path):
	roidb_file =np.load(roidb_path,allow_pickle=True)
	key = list(roidb_file.keys())[0]
	roidb_temp = roidb_file[key]
	roidb = roidb_temp[()]
	return roidb

def compute_iou(box, proposal):
	len_proposal = np.shape(proposal)[0]
	IoU = np.empty([len_proposal,1])
	for i in range(len_proposal):
		xA = max(box[0], proposal[i,0])
		yA = max(box[1], proposal[i,1])
		xB = min(box[2], proposal[i,2])
		yB = min(box[3], proposal[i,3])

		if xB<xA or yB<yA:
			IoU[i,0]=0
		else:
			area_I = (xB - xA) * (yB - yA)
			area1 = (box[2] - box[0])*(box[3] - box[1])
			area2 = (proposal[i,2] - proposal[i,0])*(proposal[i,3] - proposal[i,1])
			IoU[i,0] = area_I/float(area1 + area2 - area_I)
	return IoU


def generate_phrase_box(sbox, obox):
	N_box = len(sbox)
	phrase = np.zeros([N_box,4])
	for i in range(N_box):
		phrase[i,0] = min(sbox[i,0], obox[i,0])
		phrase[i,1] = min(sbox[i,1], obox[i,1])
		phrase[i,2] = max(sbox[i,2], obox[i,2])
		phrase[i,3] = max(sbox[i,3], obox[i,3])
	return phrase

def compute_iou_each(box1, box2):
	xA = max(box1[0], box2[0])
	yA = max(box1[1], box2[1])
	xB = min(box1[2], box2[2])
	yB = min(box1[3], box2[3])
	if xB<xA or yB<yA:
		IoU = 0
	else:
		area_I = (xB - xA) * (yB - yA)
		area1 = (box1[2] - box1[0])*(box1[3] - box1[1])
		area2 = (box2[2] - box2[0])*(box2[3] - box2[1])
		IoU = area_I/float(area1 + area2 - area_I)
	return IoU

def im_preprocess(image_path):
	image = cv2.imread(image_path)
	im_orig = image.astype(np.float32, copy=True)
	im_orig -= np.array([[[102.9801, 115.9465, 122.7717]]])
	im_shape = im_orig.shape
	im_size_min = np.min(im_shape[0:2])
	im_size_max = np.max(im_shape[0:2])
	target_size = cfg.IM_SIZE
	max_size = cfg.IM_MAX_SIZE
	im_scale = float(target_size) / float(im_size_min)
	if np.round(im_scale * im_size_max) > max_size:
		im_scale = float(max_size) / float(im_size_max)
	im = cv2.resize(im_orig, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
	im_shape_new = np.shape(im)
	im_use = np.zeros([1,im_shape_new[0], im_shape_new[1], im_shape_new[2]])
	im_use[0,:,:,:] = im
	return im_use, im_scale
########################## spatial_vector     ###################################
def spatial_vector(sbbox, obbox, N_each_batch):
	spatial_vector = np.zeros([N_each_batch, 26])
	ubbox = generate_phrase_box(sbbox, obbox)

	for i in range(N_each_batch):
		x_s = (sbbox[i][0] + sbbox[i][2]) / 2.0
		y_s = (sbbox[i][1] + sbbox[i][3]) / 2.0
		w_s = sbbox[i][2] - sbbox[i][0]
		h_s = sbbox[i][3] - sbbox[i][1]

		x_o = (obbox[i][0] + obbox[i][2]) / 2.0
		y_o = (obbox[i][1] + obbox[i][3]) / 2.0
		w_o = obbox[i][2] - obbox[i][0]
		h_o = obbox[i][3] - obbox[i][1]

		x_u = (ubbox[i][0] + ubbox[i][2]) / 2.0
		y_u = (ubbox[i][1] + ubbox[i][3]) / 2.0
		w_u = ubbox[i][2] - ubbox[i][0]
		h_u = ubbox[i][3] - ubbox[i][1]

		spatial_vector[i] = [x_s, y_s, w_s / w_o, h_s / h_o, ((w_s * h_s) / (w_o * h_o)),
							(x_s - x_o) / w_o, (y_s - y_o) / h_o, w_s / 600, h_s / 600,
							np.log(w_s / w_o), np.log(h_s / h_o),

							x_u, y_u, w_s / w_u, h_s / h_u, (w_s * h_s) / (w_u * h_u),
							(x_s - x_u) / w_u, (y_s - y_u) / h_u, w_u / 600, h_u / 600,
							np.log(w_s / w_u), np.log(h_s / h_u),

							(sbbox[i][0] - ubbox[i][0]) / w_u, (sbbox[i][1] - ubbox[i][1]) / h_u,
							(sbbox[i][2] - ubbox[i][2]) / w_u, (sbbox[i][3] - ubbox[i][3]) / h_u]
	return spatial_vector
############################     IoU      ####################################
def compute_iou_batch(sbbox, obbox, N_each_batch):
	iou = np.zeros([N_each_batch, 4])

	for i in range(N_each_batch):
		xA = max(sbbox[i][0], obbox[i][0])
		yA = max(sbbox[i][1], obbox[i][1])
		xB = min(sbbox[i][2], obbox[i][2])
		yB = min(sbbox[i][3], obbox[i][3])

		interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
		sbbox_Area = (sbbox[i][2] - sbbox[i][0] + 1) * (sbbox[i][3] - sbbox[i][1] + 1)
		obbox_Area = (obbox[i][2] - obbox[i][0] + 1) * (obbox[i][3] - obbox[i][1] + 1)
		iou[i] = interArea / float(sbbox_Area + obbox_Area - interArea)
	return iou
#####################	co_occurency	##############################
def co_occurency(sub_gt,ob_gt,N_each_batch):
	co_occurency_tensor = read_roidb(cfg.DIR + 'input/co_occurency_tensor.npz')
	P_op = read_roidb(cfg.DIR + 'input/P_op.npz')
	P_sp = read_roidb(cfg.DIR + 'input/P_sp.npz')
	P_p = read_roidb(cfg.DIR + 'input/P_p.npz')

	co_occurency_list = np.zeros((30,70))
	for i in range(N_each_batch):
		sub_idx = sub_gt[i]
		ob_idx = ob_gt[i]
		co_occurency_temp = np.zeros((70))

		for j in range (70):
			PMI = np.log10(((co_occurency_tensor[sub_idx][ob_idx][j]) * P_p[j]) / (P_sp[sub_idx][j] * P_op[ob_idx][j]))
			if PMI > 0 :
				co_occurency_temp[j] = PMI
			else:
				co_occurency_temp[j] = 0

		co_occurency_list[i] = co_occurency_temp

	return co_occurency_list
##################################################
def get_blob_pred(roidb_use, im_scale, index_sp, N_each_batch, batch_id):
	blob = {}
	sub_box = roidb_use['sub_box_gt'] * im_scale
	obj_box = roidb_use['obj_box_gt'] * im_scale
	rela = np.int32(roidb_use['rela_gt'])
	index = roidb_use['index_pred']
	sub_elmo = roidb_use['sub_elmo_gt']
	ob_elmo = roidb_use['ob_elmo_gt']
	sub_gt = np.int32(roidb_use['sub_gt'])
	ob_gt = np.int32(roidb_use['obj_gt'])
	index_use = index[batch_id * N_each_batch: (batch_id + 1) * N_each_batch]
	sub_gt_use = sub_gt[index_use]
	ob_gt_use = ob_gt[index_use]
	sub_box_use = sub_box[index_use, :]
	obj_box_use = obj_box[index_use, :]
	rela_use = rela[index_use]
	sub_elmo_use = sub_elmo[index_use]
	ob_elmo_use = ob_elmo[index_use]
	co_occurency_log = co_occurency(sub_gt_use, ob_gt_use, N_each_batch)
	blob['sub_label'] = sub_gt_use
	blob['obj_label'] = ob_gt_use
	blob['sub_box'] = sub_box_use
	blob['obj_box'] = obj_box_use
	blob['rela'] = rela_use
	blob['sub_elmo'] = sub_elmo_use
	blob['ob_elmo'] = ob_elmo_use
	blob['iou'] = compute_iou_batch(sub_box_use, obj_box_use, N_each_batch)
	blob['union_box'] = generate_phrase_box(sub_box_use, obj_box_use)
	blob['sub_spatial_vector'] = spatial_vector(sub_box_use, obj_box_use, N_each_batch)
	blob['ob_spatial_vector'] = spatial_vector(obj_box_use, sub_box_use, N_each_batch)
	blob['co_occurency_tensor'] = co_occurency_log

	return blob
############################################################################################################
#________________________________________________________________________________________________________#
class VSimplE(object):
	def __init__(self):
		self.predictions = {}
		self.losses = {}
		self.layers = {}
		self.feat_stride = [16, ]
		self.scope = 'vgg_16'

	def create_graph(self, N_each_batch, index_sp, index_cls, num_classes, num_predicates):
		self.image = tf.placeholder(tf.float32, 		shape=[1, None, None, 3])
		self.sbox = tf.placeholder(tf.float32, 			shape=[N_each_batch, 4])
		self.obox = tf.placeholder(tf.float32, 			shape=[N_each_batch, 4])
		self.sub_elmo = tf.placeholder(tf.float32 , 	shape=[N_each_batch , 128] )
		self.ob_elmo = tf.placeholder(tf.float32 , 		shape=[N_each_batch , 128] )
		self.iou = tf.placeholder(tf.float32 , 			shape=[N_each_batch , 4] )
		self.union_box = tf.placeholder(tf.float32 , 	shape=[N_each_batch , 4] )
		self.sub_sp = tf.placeholder(tf.float32 , 		shape=[N_each_batch , 26] )
		self.ob_sp = tf.placeholder(tf.float32 , 		shape=[N_each_batch , 26] )
		self.co_occurency_log = tf.placeholder(tf.float32,shape=[N_each_batch , 70] )
		self.rela_label = tf.placeholder(tf.int32, 		shape=[N_each_batch,])
		self.keep_prob = tf.placeholder(tf.float32)
  
		self.sub_label = tf.placeholder(tf.int32, shape=[N_each_batch, ])
		self.obj_label = tf.placeholder(tf.int32, shape=[N_each_batch, ])
		self.sub_obj_p = tf.placeholder(tf.float32, shape=[None, num_predicates])
		self.object_word = np.load(cfg.DIR + '/input/oList_word_embedding.npy')
		self.robject = np.load(cfg.DIR + '/input/rList_word_embedding.npy')
		conf = np.load(cfg.DIR + '/input/language_inter.npz')
		self.sub_obj_pred = conf['sub_obj']
		self.sub_pred = conf['sub']
		self.obj_pred = conf['obj']

		self.index_sp = index_sp
		self.index_cls = index_cls
		self.num_classes = num_classes
		self.num_predicates = num_predicates
		self.N_each_batch = N_each_batch
		self.build_dete_network()
		self.setup_ent_emb()
		self.build_rd_network()
		self.add_rd_loss()
# **********************************************		VGG 16 		*************************************************
	def build_dete_network(self, is_training=True):
		net_conv = self.image_to_head(is_training)
		sub_pool5 = self.crop_pool_layer(net_conv, self.sbox, "sub_pool5")
		ob_pool5 = self.crop_pool_layer(net_conv, self.obox, "ob_pool5")
		sub_fc7 = self.head_to_tail(sub_pool5, is_training, reuse = False)#faster_rcnn
		ob_fc7 = self.head_to_tail(ob_pool5, is_training, reuse = True)#faster_rcnn
		with tf.variable_scope(self.scope, self.scope):
			sub_cls_prob, sub_cls_pred = self.region_classification(sub_fc7, is_training, reuse = False)
		with tf.variable_scope(self.scope, self.scope):
			ob_cls_prob, ob_cls_pred = self.region_classification(ob_fc7, is_training, reuse = True)

		self.predictions['sub_cls_prob'] = sub_cls_prob
		self.predictions['sub_cls_pred'] = sub_cls_pred
		self.predictions['ob_cls_prob'] = ob_cls_prob
		self.predictions['ob_cls_pred'] = ob_cls_pred
		self.layers['sub_pool5'] = sub_pool5
		self.layers['ob_pool5'] = ob_pool5
		self.layers['sub_fc7'] = sub_fc7
		self.layers['ob_fc7'] = ob_fc7

	def image_to_head(self, is_training, reuse=False):
		with tf.variable_scope(self.scope, self.scope, reuse=reuse):
			net = slim.repeat(self.image, 2, slim.conv2d, 64, [3, 3],trainable=is_training, scope='conv1')
			net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool1')
			net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3],trainable=is_training, scope='conv2')
			net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool2')
			net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3],trainable=is_training, scope='conv3')
			net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool3')
			net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3],trainable=is_training, scope='conv4')
			net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool4')
			net_conv = slim.repeat(net, 3, slim.conv2d, 512, [3, 3],trainable=is_training, scope='conv5')
			self.layers['head'] = net_conv
			return net_conv

	def head_to_tail(self, pool5, is_training, reuse=False):
		with tf.variable_scope(self.scope, self.scope, reuse=reuse):
			pool5_flat = slim.flatten(pool5, scope='flatten')
			fc6 = slim.fully_connected(pool5_flat, 4096, scope='fc6')
			fc6 = slim.dropout(fc6, keep_prob=self.keep_prob, is_training=True, scope='dropout6')
			fc7 = slim.fully_connected(fc6, 4096, scope='fc7')
			fc7 = slim.dropout(fc7, keep_prob=self.keep_prob, is_training=True, scope='dropout7')
			return fc7

	def crop_pool_layer(self, bottom, rois, name):
		with tf.variable_scope(name) as scope:
			n=tf.to_int32(rois.shape[0])
			batch_ids = tf.zeros([n,],dtype=tf.int32)
			bottom_shape = tf.shape(bottom)
			height = (tf.to_float(bottom_shape[1]) - 1.) * np.float32(self.feat_stride[0])
			width = (tf.to_float(bottom_shape[2]) - 1.) * np.float32(self.feat_stride[0])
			x1 = tf.slice(rois, [0, 0], [-1, 1], name="x1") / width
			y1 = tf.slice(rois, [0, 1], [-1, 1], name="y1") / height
			x2 = tf.slice(rois, [0, 2], [-1, 1], name="x2") / width
			y2 = tf.slice(rois, [0, 3], [-1, 1], name="y2") / height
			# Won't be back-propagated to rois anyway, but to save time
			bboxes = tf.stop_gradient(tf.concat([y1, x1, y2, x2], 1))
			crops = tf.image.crop_and_resize(bottom, bboxes, tf.to_int32(batch_ids), [cfg.POOLING_SIZE*2, cfg.POOLING_SIZE*2], method='bilinear',name="crops")
			pooling = max_pool(crops, 2, 2, 2, 2, name="max_pooling")
		return pooling

	def region_classification(self, fc7, is_training, reuse = False):
		cls_score = slim.fully_connected(fc7, self.num_classes, activation_fn=None, scope='cls_score', reuse=reuse)
		cls_prob = tf.nn.softmax(cls_score, name="cls_prob")
		cls_pred = tf.argmax(cls_score, axis=1, name="cls_pred")
		return cls_prob, cls_pred

############################################      Relationship detection          ###########################################################
	def setup_ent_emb(self):
		self.head = tf.get_variable(name="RD_head_emb",initializer=tf.random_uniform(shape=[30, 2000], minval=0, maxval=1999,dtype=tf.int32))
		self.tail = tf.get_variable(name="RD_tail_emb",initializer=tf.random_uniform(shape=[30, 2000], minval=0, maxval=1999,dtype=tf.int32))

	def Bi_GRU(self, input, name):
		with tf.variable_scope(name):
			fw_gru_cell = tf.contrib.rnn.GRUCell(1000)
			bw_gru_cell = tf.contrib.rnn.GRUCell(1000)
			(fw_outputs, bw_outputs),_ = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_gru_cell,cell_bw=bw_gru_cell,inputs=input,dtype=tf.float32)
			outputs = tf.concat((fw_outputs, bw_outputs), 2)
			outputs = tf.reshape(outputs,[30,2000])
			return outputs

	def build_rd_network(self):
		# _________________________Sub_feature
		sub_cls_prob = self.predictions['sub_cls_prob']
		sub_cls_pred = self.predictions['sub_cls_pred']
		sub_fc = self.layers['sub_fc7']
		sbox = self.sbox
		sub_sp = self.sub_sp
		# _________________________Obj_feature
		ob_cls_prob = self.predictions['ob_cls_prob']
		ob_cls_pred = self.predictions['ob_cls_pred']
		ob_fc = self.layers['ob_fc7']
		obox = self.obox
		ob_sp = self.ob_sp
		# _____________________________Spatial
		co_occur = self.co_occurency_log
		union_box = self.union_box
		iou = self.iou
		sub_sp_info = tf.concat([iou, sbox, sub_sp, union_box], axis=1)
		ob_sp_info = tf.concat([iou, obox, ob_sp, union_box], axis=1)
		# ________________________________Word2Vec
		vector_dic = tf.Variable(self.object_word, trainable=False, name='VD_vo')
		sub_onehot = tf.matmul(tf.one_hot(self.sub_label, self.num_classes - 1), vector_dic)
		obj_onehot = tf.matmul(tf.one_hot(self.obj_label, self.num_classes - 1), vector_dic)
		# ______________________________________________________4. Balanced Size of extracted features
		sub_sp_info = slim.fully_connected(sub_sp_info, 1000, activation_fn=tf.nn.relu, scope='RD_subinfo_fc')
		sub_cls_prob = slim.fully_connected(sub_cls_prob, 500, activation_fn=tf.nn.relu, scope='RD_subclsprob_fc')
		sub_fc = slim.fully_connected(sub_fc, 3000, activation_fn=tf.nn.relu, scope='RD_subfc_fc')
		label_s = slim.fully_connected(sub_onehot, 1000, activation_fn=tf.nn.relu, scope='RD_ls1')

		ob_sp_info = slim.fully_connected(ob_sp_info, 1000, activation_fn=tf.nn.relu, scope='RD_obinfo_fc')
		ob_cls_prob = slim.fully_connected(ob_cls_prob, 500, activation_fn=tf.nn.relu, scope='RD_obclsprob_fc')
		ob_fc = slim.fully_connected(ob_fc, 3000, activation_fn=tf.nn.relu, scope='RD_obfc_fc')
		label_o = slim.fully_connected(obj_onehot, 1000, activation_fn=tf.nn.relu, scope='RD_lo1')
		if self.index_sp:
			sub_fc = tf.concat([sub_fc, label_s, sub_sp_info, sub_cls_prob], axis=1)
			ob_fc = tf.concat([ob_fc, label_o, ob_sp_info, ob_cls_prob], axis=1)
		if self.index_cls:
			sub_fc = tf.concat([sub_fc, label_s, sub_sp_info, sub_cls_prob], axis=1)
			ob_fc = tf.concat([ob_fc, label_o, ob_sp_info, ob_cls_prob], axis=1)
		# ________________________________________Embedding space
		sub_fc = slim.fully_connected(sub_fc, 2000, activation_fn=tf.nn.relu, scope='RD_sub_fc')
		ob_fc  = slim.fully_connected(ob_fc , 2000, activation_fn=tf.nn.relu, scope='RD_ob_fc' )
		# __________________________________________________________________Sampling Head and tail       
		s = tf.shape(self.head)
		batch_idx = tf.tile(tf.expand_dims(tf.range(s[0]), 1), [1, s[1]])
		sub_head = tf.gather_nd(params=sub_fc, indices=tf.stack([batch_idx, self.head], axis=-1), name='RD_sub_head')
		sub_tail = tf.gather_nd(params=sub_fc, indices=tf.stack([batch_idx, self.tail], axis=-1), name='RD_sub_tail')
		ob_head  = tf.gather_nd(params=ob_fc,  indices=tf.stack([batch_idx, self.head], axis=-1), name='RD_ob_head')
		ob_tail  = tf.gather_nd(params=ob_fc,  indices=tf.stack([batch_idx, self.tail], axis=-1), name='RD_ob_tail')

		sub_head = slim.fully_connected(sub_head, 2000, activation_fn=tf.nn.relu, scope='RD_subhead_fc')
		ob_head  = slim.fully_connected(ob_head , 2000, activation_fn=tf.nn.relu, scope='RD_obhead_fc')
		sub_tail = slim.fully_connected(sub_tail, 2000, activation_fn=tf.nn.relu, scope='RD_subtail_fc')
		ob_tail  = slim.fully_connected(ob_tail , 2000, activation_fn=tf.nn.relu, scope='RD_obtail_fc')

		# ___________________________________1.VSimplE 
		vsimple_vec = ((ob_tail - sub_head) + (sub_tail - ob_head)) / 2.0
		# _________________________________________________________________________      Internal Language
		# sub_obj_p = slim.fully_connected(self.sub_obj_p, 500, activation_fn=tf.nn.relu, scope='RD_l1', reuse=False)
		# vsimple_vec = tf.concat([vsimple_vec, sub_obj_p], axis=1)
		vsimple_vec = slim.fully_connected(vsimple_vec, 2000, activation_fn=tf.nn.relu ,scope='RD_vsimple')
		# ___________________________________________________________________6.Bi_GRU
		gru_vec = tf.concat([sub_fc, vsimple_vec, ob_fc], axis=1)
		gru_vec = slim.fully_connected(gru_vec, 2000, activation_fn=tf.nn.relu, scope='RD_fc_GRU')
		gru_vec = tf.expand_dims(gru_vec, 0)
		gru_vec = self.Bi_GRU(gru_vec, "RD_GRU")

		PMI_score   = slim.fully_connected(co_occur   , 500, activation_fn=tf.nn.relu, scope='RD_fc1')
		vsimple_vec = slim.fully_connected(vsimple_vec, 500, activation_fn=tf.nn.relu, scope='RD_fc2')
		gru_vec     = slim.fully_connected(gru_vec    , 500, activation_fn=tf.nn.relu, scope='RD_fc3')

		# _________________________________________________________________13.Score of VsimplE & GRU
		vsimple_score = slim.fully_connected(vsimple_vec, self.num_predicates, activation_fn=None, scope='RD_vsimple_score')
		gru_score     = slim.fully_connected(gru_vec    , self.num_predicates, activation_fn=None, scope='RD_gru_score')
		self.layers['vsimple_score'] = vsimple_score
		self.layers['gru_score']     = gru_score
		# ___________________________________________________________________7. Feature Fusion
		rela_vec    = tf.concat([PMI_score, vsimple_vec, gru_vec], axis=1)
		rela_vec    = slim.fully_connected(rela_vec, 500, activation_fn=tf.nn.relu, scope='RD_fc4')
		rela_score  = slim.fully_connected(rela_vec, self.num_predicates, activation_fn=None, scope='RD_fc5')
		self.layers['rela_score'] = rela_score
		################################________________MULTI Class CALSSIFICATION___________________#############################################
		rela_prob = tf.nn.softmax(rela_score)
		self.layers['rela_prob'] = rela_prob
		##############################__________________MULTI LABEL CALSSIFICATION___________________#############################################
		#self.predictions['rela_pred_all'] = tf.concat([tf.nn.sigmoid(rela_score)], axis = 1)
		#rela_label 	= tf.one_hot( self.rela_label, self.num_predicates)
		#self.layers['rd_loss'] = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits( labels=rela_label, logits=rela_score))


	def add_rd_loss(self):
		rela_score = self.layers['rela_score']
		rela_prob = self.layers['rela_prob']
		rela_label = self.rela_label
		rd_loss = tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits(labels = rela_label, logits = rela_score) )
		self.losses['rd_loss'] = rd_loss
		acc_each = tf.nn.in_top_k(rela_score, rela_label, 1)
		self.losses['acc_each'] = acc_each
		self.losses['acc'] = tf.reduce_mean( tf.cast(acc_each, tf.float32) )
		rela_pred = tf.argmax(rela_score, 1)
		self.predictions['rela_pred'] = rela_pred
		rela_max_prob = tf.reduce_max(rela_prob, 1)
		self.predictions['rela_max_prob'] = rela_max_prob

	def test_predicate(self, sess, roidb_use):
		im, im_scale = im_preprocess(roidb_use['image'])
		batch_num = len(roidb_use['index_pred'])/self.N_each_batch
		pred_rela = np.zeros([len(roidb_use['index_pred']),])
		pred_rela_score = np.zeros([len(roidb_use['index_pred']),])

		for batch_id in range(np.int32(batch_num)):
			blob = get_blob_pred(roidb_use, im_scale, self.index_sp, self.N_each_batch, batch_id)
			feed_dict = {self.image: im, self.co_occurency_log : blob['co_occurency_tensor'],
						self.sbox: blob['sub_box'], self.obox: blob['obj_box'] ,
						self.sub_elmo : blob['sub_elmo'] ,self.ob_elmo: blob['ob_elmo'],
						self.iou : blob['iou'] , self.union_box : blob['union_box'],
						self.sub_sp : blob['sub_spatial_vector'] ,self.ob_sp:blob['ob_spatial_vector'],
						self.rela_label: blob['rela'],self.keep_prob: 0.5,
						self.obj_label: blob['obj_label'],self.sub_label: blob['sub_label'],
						self.sub_obj_p: self.sub_obj_pred[blob['sub_label'], blob['obj_label']]
						}

			predictions = sess.run(self.predictions, feed_dict = feed_dict)
			pred_rela[batch_id*self.N_each_batch:(batch_id+1)*self.N_each_batch] = predictions['rela_pred'][:]
			pred_rela_score[batch_id*self.N_each_batch:(batch_id+1)*self.N_each_batch] = predictions['rela_max_prob'][:]
		N_rela = len(roidb_use['rela_gt'])
		pred_rela = pred_rela[0:N_rela]
		pred_rela_score = pred_rela_score[0:N_rela]
		return pred_rela, pred_rela_score

def max_pool(x, h, w, s_y, s_x, name, padding='SAME'):
	return tf.nn.max_pool(x, ksize=[1,h,w,1], strides=[1, s_x, s_y, 1], padding=padding, name=name)
#_______________________________________________________________________________________________________________
N_cls = cfg.VRD_NUM_CLASS
N_rela = cfg.VRD_NUM_RELA
N_each_batch = cfg.VRD_BATCH_NUM

index_sp = False
index_cls = False
tf.reset_default_graph()
vnet = VSimplE()
vnet.create_graph(N_each_batch, index_sp, index_cls, N_cls, N_rela)
roidb_path = cfg.DIR + 'input/vrd_roidb.npz'
# reader = pywrap_tensorflow.NewCheckpointReader(model_path)
# var_to_shape_map = reader.get_variable_to_shape_map()
# for key in sorted(var_to_shape_map):
#     print("tensor_name: ", key)
roidb_read = read_roidb(roidb_path)
train_roidb = roidb_read['train_roidb']
test_roidb = roidb_read['test_roidb']
N_train = len(train_roidb)
N_test = len(test_roidb)
saver = tf.train.Saver()
model_path = cfg.DIR + 'Model_ckpt/Pred0010.ckpt'
with tf.Session() as sess:
	init = tf.global_variables_initializer()
	sess.run(init)
	saver.restore(sess, model_path)
	print('*****************************  Epoch:1 >>> Pred0010.ckpt	*****************************')
	pred_roidb = []
	for roidb_id in range(N_test):
		if (roidb_id+1)%100 == 0:
			print("Number of Tested: ",roidb_id + 1)
		roidb_use = test_roidb[roidb_id]
		if len(roidb_use['rela_gt']) == 0:
			pred_roidb.append({})
			continue
		pred_rela, pred_rela_score = vnet.test_predicate(sess, roidb_use)
		pred_roidb_temp = {'pred_rela': pred_rela, 'pred_rela_score': pred_rela_score,
							'sub_box_dete': roidb_use['sub_box_gt'], 'obj_box_dete': roidb_use['obj_box_gt'],
							'sub_dete': roidb_use['sub_gt'], 'obj_dete': roidb_use['obj_gt']}
		pred_roidb.append(pred_roidb_temp)
roidb = {}
roidb['pred_roidb'] = pred_roidb
print("Test is Done!")
#________________________________________________
gt_roidb_path = cfg.DIR + 'input/vrd_roidb.npz'
roidb_read = read_roidb(gt_roidb_path)
train_roidb = roidb_read['train_roidb']
test_roidb = roidb_read['test_roidb']
pred_roidb = roidb['pred_roidb']
R50, num_right50 = rela_recall(test_roidb, pred_roidb, 50)
R100, num_right100 = rela_recall(test_roidb, pred_roidb, 100)
print('Predicate prediction performances on VRD :	R@ 50: {0},   R@ 100: {1}'.format(round(R50*100 , 2), round(R100*100 , 2)))